In [12]:
#Multi label classfication
import jieba
import csv
from sklearn.datasets.base import Bunch
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

bunch = Bunch(target_name=[],label=[],filenames=[],contents=[])    
catelist = [-2,-1,0,1,2]
bunch.target_name.extend(catelist)
jieba.load_userdict("dictionary.txt")
stopWords=[]

# 讀入停用詞檔
with open('stopWords.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopWords.append(data)

with open(r'G:\報告+作業\IR\train.csv',encoding='big5', newline='') as csvfile:

  # 讀取 CSV 檔案內容
    rows = csv.reader(csvfile)
    rows.__next__()
  # 以迴圈輸出每一列
    for row in rows:
    
        seg_list = jieba.cut(row[2])
        #print("jieba分詞："," ".join(seg_list))
        
        #移除停用詞
        remainderWords = list(filter(lambda a: a not in stopWords and a != '\n', seg_list))
        #print("移除停用詞："," ".join(remainderWords))
        
        bunch.label.append(row[3]) #class
        bunch.filenames.append(row[0]) #docID
        bunch.contents.append(' '.join(remainderWords).encode()) #內容

#构造TF-IDF词向量空间对象
tfidfspace = Bunch(target_name=bunch.target_name,label=bunch.label,
                   filenames = bunch.filenames,tdm=[],vocabulary={})

#使用TfidVectorizer初始化向量空间模型
vectorizer = TfidfVectorizer(sublinear_tf=True,max_df=0.5)
transformer = TfidfTransformer() #该类会统计每个词语的TF-IDF权值

#文本转为词频矩阵，单独保存字典文件
tfidfspace.tdm = vectorizer.fit_transform(bunch.contents)
tfidfspace.vocabulary = vectorizer.vocabulary_
#print(tfidfspace.vocabulary)


#应用朴素贝叶斯算法
#alpha:0.001 alpha越小，迭代次数越多，精度越高

# X -> features, y -> label 
X = tfidfspace.tdm
y = tfidfspace.label

#feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

X_new = SelectKBest(chi2, k=1500).fit_transform(X, y)
  
# dividing X, y into train (75%) and test data (25%)
X_train, X_test, y_train, y_test = train_test_split(X_new, y, random_state = 0) 

# training a Naive Bayes classifier 
clf = MultinomialNB(alpha= 0.001).fit(X_train, y_train) 
clf_predictions = clf.predict(X_test) 
  
# accuracy on X_test 
accuracy = clf.score(X_test, y_test) 
print(accuracy) 
print("classes: "," ".join(clf.classes_))
# creating a confusion matrix 
cm = confusion_matrix(y_test, clf_predictions) 
print(cm)

report = metrics.classification_report(y_test,clf_predictions)
print(report)

0.712
classes:  -1 -2 0 1 2
[[ 328   15  269    2    0]
 [  32   13   72    4    4]
 [  61   12 1064   13    0]
 [   6    1   82   19    1]
 [   0    0    2    0    0]]
              precision    recall  f1-score   support

          -1       0.77      0.53      0.63       614
          -2       0.32      0.10      0.16       125
           0       0.71      0.93      0.81      1150
           1       0.50      0.17      0.26       109
           2       0.00      0.00      0.00         2

    accuracy                           0.71      2000
   macro avg       0.46      0.35      0.37      2000
weighted avg       0.69      0.71      0.68      2000



In [14]:
# training a KNN classifier 
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 7).fit(X_train, y_train) 
  
# accuracy on X_test 
accuracy = knn.score(X_test, y_test) 
print(accuracy) 
  
# creating a confusion matrix 
knn_predictions = knn.predict(X_test)  
cm = confusion_matrix(y_test, knn_predictions) 
print(cm)

report = metrics.classification_report(y_test,knn_predictions)
print(report)

0.6675
[[ 243    2  369    0    0]
 [  24    6   95    0    0]
 [  67    1 1081    1    0]
 [   4    0  100    5    0]
 [   0    0    2    0    0]]
              precision    recall  f1-score   support

          -1       0.72      0.40      0.51       614
          -2       0.67      0.05      0.09       125
           0       0.66      0.94      0.77      1150
           1       0.83      0.05      0.09       109
           2       0.00      0.00      0.00         2

    accuracy                           0.67      2000
   macro avg       0.58      0.29      0.29      2000
weighted avg       0.69      0.67      0.61      2000



c:\users\master\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
# training a linear SVM classifier 
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train) 
svm_predictions = svm_model_linear.predict(X_test) 

# model accuracy for X_test   
accuracy = svm_model_linear.score(X_test, y_test) 
print(accuracy)
  
# creating a confusion matrix 
cm = confusion_matrix(y_test, svm_predictions) 
print(cm)

report = metrics.classification_report(y_test,svm_predictions)
print(report)

0.6955
[[ 267    3  344    0    0]
 [  18    9   98    0    0]
 [  46    2 1099    3    0]
 [   5    0   88   16    0]
 [   1    0    1    0    0]]
              precision    recall  f1-score   support

          -1       0.79      0.43      0.56       614
          -2       0.64      0.07      0.13       125
           0       0.67      0.96      0.79      1150
           1       0.84      0.15      0.25       109
           2       0.00      0.00      0.00         2

    accuracy                           0.70      2000
   macro avg       0.59      0.32      0.35      2000
weighted avg       0.72      0.70      0.65      2000



c:\users\master\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
# training a DescisionTreeClassifier 
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(max_depth = 40).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 
  
# creating a confusion matrix 
cm = confusion_matrix(y_test, dtree_predictions) 
print(cm)

report = metrics.classification_report(y_test,dtree_predictions)
print(report)

[[ 194   10  408    2    0]
 [  15   10   99    1    0]
 [  34    2 1113    1    0]
 [   3    2   97    7    0]
 [   1    0    1    0    0]]
              precision    recall  f1-score   support

          -1       0.79      0.32      0.45       614
          -2       0.42      0.08      0.13       125
           0       0.65      0.97      0.78      1150
           1       0.64      0.06      0.12       109
           2       0.00      0.00      0.00         2

    accuracy                           0.66      2000
   macro avg       0.50      0.29      0.30      2000
weighted avg       0.67      0.66      0.60      2000



c:\users\master\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [77]:
#Multi label classfication
import jieba
import csv
from sklearn.datasets.base import Bunch
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

bunch = Bunch(target_name=[],label=[],filenames=[],contents=[])    
catelist = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
bunch.target_name.extend(catelist)
jieba.load_userdict("dictionary.txt")

with open(r'G:\報告+作業\IR\train.csv',encoding='big5', newline='') as csvfile:

  # 讀取 CSV 檔案內容
    rows = csv.reader(csvfile)
    rows.__next__()
  # 以迴圈輸出每一列
    for row in rows:
    
        seg_list = jieba.cut(row[2])
        #print("jieba分詞："," ".join(seg_list))
    
        bunch.filenames.append(row[0]) #docID
        bunch.contents.append(' '.join(seg_list).encode()) #內容
        labels = []
        for label in row[4:]:
            labels.append(int(label))
        bunch.label.append(labels) #labels

# X -> features, y -> label 
X = bunch.contents
y = bunch.label

# dividing X, y into train (75%) and test data (25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0) 

from sklearn.preprocessing import MultiLabelBinarizer
# transform y into a matrix
mb = MultiLabelBinarizer(classes = [1,2,3,4,5,6,7,8,9,10,11,12,13,14])
y_train = mb.fit_transform(y_train)
y_test = mb.fit_transform(y_test)

#using OneVsRestClassifier
print("using OneVsRestClassifier")
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
classifier = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC()))])

# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)
#print(predictions)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))

all_labels = mb.inverse_transform(predictions)
for item, labels in zip(X_test, all_labels):
    print('{0} => {1}'.format(str(item,'utf-8'), ', '.join('%s' %id for id in labels)))

c:\users\master\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\_label.py:987: UserWarning: unknown class(es) [0] will be ignored
  .format(sorted(unknown, key=str)))


using OneVsRestClassifier
Accuracy =  0.748
不演了 不演了   政治 清流 白色力量 ww => 
說個笑話 ： 睡菊 是 五星市長 … 大菊 為重 => 10
 => 
回收黨 看上 中 國瑜 了   有趣 => 12
不 是 一 堆 柯糞 聽到 柯韓一家親 就 馬上 說 蟑螂 硬要 把 柯 跟 韓放 一起 講 很 噁心 嗎 柯糞 自己 去死 一 死好 嗎 zz => 2, 4
導遊 部 分 薪水 就是 從 購物 站 抽成 => 
連 這 種 都 賴帳 真的很 廢 => 
這樣 也 算 切割 ?   直接 講 我 不 認得 她 不 是 更 猛 => 
雖 敗猶榮 => 
若 國民黨 繼續執政 ， 終有一天 台灣 會 被 統一 => 
推 => 
舔共 的 還 在 台灣 趴 趴 走   才 悲哀 吧   ： ） => 
發現 民眾黨 往 正常 方向 移動 ， 只好 三角形 拉往 不 正常 這邊 來 => 
真的   主權 問題 不 敢生 小孩 覺得 有 點扯 => 
高雄市 隨便 丟 兩 張圖 出來 就 說 卡韓 。 => 4
TMD 主席 不 是 柯屁 嗎 ?   不關 他 屁事 ? => 2, 12
柯韓 一起 下 吧   高雄市 整年 都 在 搞 選舉 自稱 務實 的 市長 在 護航   務實 在 哪 ？ 就 政治 算計 而已 => 2
你看 職棒 簽賭 的 組頭 不 都 藍營 的 嗎 => 2
https : / / i . imgur . com / BIlyq5q . jpg => 
反正 我 不會 投 你 => 
紅利 用完 了     中共 經濟 專家 都 說 保 4 爭 5 了 明年 保 4 爭 5     勢在必行 => 
柯 鎮惡 的 風向 也 變換 得 太 快 了 => 
不 穩 才 有加 錢 ， 賺錢 領導 越維越 不 穩 => 
是 購物 旅行 ... 導遊 會 帶 你們 去 買買 買 但是 最後一天   要包 大 紅包 塞給 導遊 和 司機 ZZ 是 我 親戚 的 經驗   以前 常去   後 來近 幾年 不再 去 => 
是不是 要 開除黨籍 => 
阿諾 人 呢 => 
還好 我 的 醫材 都 銷美 歐   蚵蚵 簡單 啊   TSMC 東西 +1 00%   蚵蚵 聽說 TSMC 產能 

In [63]:
# using classifier chains
print("using classifier chains")
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import GaussianNB
# initialize classifier chains multi-label classifier
# with a gaussian naive bayes base classifier
classifier = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',  ClassifierChain(GaussianNB()))])

# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)
#print(predictions)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))

all_labels = mb.inverse_transform(predictions)
for item, labels in zip(X_test, all_labels):
    print('{0} => {1}'.format(str(item,'utf-8'), ', '.join('%s' %id for id in labels)))

using classifier chains
Accuracy =  0.3945
不演了 不演了   政治 清流 白色力量 ww => 
說個笑話 ： 睡菊 是 五星市長 … 大菊 為重 => 10
 => 1, 3, 10, 11
回收黨 看上 中 國瑜 了   有趣 => 12
不 是 一 堆 柯糞 聽到 柯韓一家親 就 馬上 說 蟑螂 硬要 把 柯 跟 韓放 一起 講 很 噁心 嗎 柯糞 自己 去死 一 死好 嗎 zz => 4
導遊 部 分 薪水 就是 從 購物 站 抽成 => 
連 這 種 都 賴帳 真的很 廢 => 1, 4, 6, 10
這樣 也 算 切割 ?   直接 講 我 不 認得 她 不 是 更 猛 => 5
雖 敗猶榮 => 
若 國民黨 繼續執政 ， 終有一天 台灣 會 被 統一 => 
推 => 1, 3, 10, 11
舔共 的 還 在 台灣 趴 趴 走   才 悲哀 吧   ： ） => 2, 4, 8
發現 民眾黨 往 正常 方向 移動 ， 只好 三角形 拉往 不 正常 這邊 來 => 
真的   主權 問題 不 敢生 小孩 覺得 有 點扯 => 
高雄市 隨便 丟 兩 張圖 出來 就 說 卡韓 。 => 4
TMD 主席 不 是 柯屁 嗎 ?   不關 他 屁事 ? => 
柯韓 一起 下 吧   高雄市 整年 都 在 搞 選舉 自稱 務實 的 市長 在 護航   務實 在 哪 ？ 就 政治 算計 而已 => 2
你看 職棒 簽賭 的 組頭 不 都 藍營 的 嗎 => 2, 4, 8, 12
https : / / i . imgur . com / BIlyq5q . jpg => 
反正 我 不會 投 你 => 2, 4, 5, 8
紅利 用完 了     中共 經濟 專家 都 說 保 4 爭 5 了 明年 保 4 爭 5     勢在必行 => 
柯 鎮惡 的 風向 也 變換 得 太 快 了 => 2, 6, 10
不 穩 才 有加 錢 ， 賺錢 領導 越維越 不 穩 => 
是 購物 旅行 ... 導遊 會 帶 你們 去 買買 買 但是 最後一天   要包 大 紅包 塞給 導遊 和 司機 ZZ 是 我 親戚 的 經驗   以前 常去   後 來近 幾年 不再 去 => 
是不是 要 開除黨籍 => 10
阿諾 人 呢 

In [64]:
# using binary relevance
print("using binary relevance")
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',  BinaryRelevance(GaussianNB()))])
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)
#print(predictions)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))

all_labels = mb.inverse_transform(predictions)
for item, labels in zip(X_test, all_labels):
    print('{0} => {1}'.format(str(item,'utf-8'), ', '.join('%s' %id for id in labels)))

using binary relevance
Accuracy =  0.3945
不演了 不演了   政治 清流 白色力量 ww => 
說個笑話 ： 睡菊 是 五星市長 … 大菊 為重 => 10
 => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14
回收黨 看上 中 國瑜 了   有趣 => 12
不 是 一 堆 柯糞 聽到 柯韓一家親 就 馬上 說 蟑螂 硬要 把 柯 跟 韓放 一起 講 很 噁心 嗎 柯糞 自己 去死 一 死好 嗎 zz => 4
導遊 部 分 薪水 就是 從 購物 站 抽成 => 
連 這 種 都 賴帳 真的很 廢 => 1, 2, 4, 6, 7, 10, 12, 14
這樣 也 算 切割 ?   直接 講 我 不 認得 她 不 是 更 猛 => 5
雖 敗猶榮 => 
若 國民黨 繼續執政 ， 終有一天 台灣 會 被 統一 => 
推 => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14
舔共 的 還 在 台灣 趴 趴 走   才 悲哀 吧   ： ） => 2, 4, 8
發現 民眾黨 往 正常 方向 移動 ， 只好 三角形 拉往 不 正常 這邊 來 => 
真的   主權 問題 不 敢生 小孩 覺得 有 點扯 => 
高雄市 隨便 丟 兩 張圖 出來 就 說 卡韓 。 => 4
TMD 主席 不 是 柯屁 嗎 ?   不關 他 屁事 ? => 
柯韓 一起 下 吧   高雄市 整年 都 在 搞 選舉 自稱 務實 的 市長 在 護航   務實 在 哪 ？ 就 政治 算計 而已 => 2
你看 職棒 簽賭 的 組頭 不 都 藍營 的 嗎 => 2, 4, 8, 12
https : / / i . imgur . com / BIlyq5q . jpg => 
反正 我 不會 投 你 => 2, 4, 5, 8, 12
紅利 用完 了     中共 經濟 專家 都 說 保 4 爭 5 了 明年 保 4 爭 5     勢在必行 => 
柯 鎮惡 的 風向 也 變換 得 太 快 了 => 2, 6, 10, 12
不 穩 才 有加 錢 ， 賺錢 領導 越維越 不 穩 => 
是 購物 旅行 ... 導遊 會 帶 你們 去 買買 買 但是 最後一天 

In [65]:
# using Label Powerset
print("using Label Powerset")
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB
# initialize Label Powerset multi-label classifier
# with a gaussian naive bayes base classifier
classifier = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',  LabelPowerset(GaussianNB()))])
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)
#print(predictions)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))

all_labels = mb.inverse_transform(predictions)
for item, labels in zip(X_test, all_labels):
    print('{0} => {1}'.format(str(item,'utf-8'), ', '.join('%s' %id for id in labels)))

using Label Powerset
Accuracy =  0.4385
不演了 不演了   政治 清流 白色力量 ww => 
說個笑話 ： 睡菊 是 五星市長 … 大菊 為重 => 10
 => 10, 12
回收黨 看上 中 國瑜 了   有趣 => 2, 12
不 是 一 堆 柯糞 聽到 柯韓一家親 就 馬上 說 蟑螂 硬要 把 柯 跟 韓放 一起 講 很 噁心 嗎 柯糞 自己 去死 一 死好 嗎 zz => 
導遊 部 分 薪水 就是 從 購物 站 抽成 => 
連 這 種 都 賴帳 真的很 廢 => 7
這樣 也 算 切割 ?   直接 講 我 不 認得 她 不 是 更 猛 => 5
雖 敗猶榮 => 
若 國民黨 繼續執政 ， 終有一天 台灣 會 被 統一 => 
推 => 10, 12
舔共 的 還 在 台灣 趴 趴 走   才 悲哀 吧   ： ） => 4
發現 民眾黨 往 正常 方向 移動 ， 只好 三角形 拉往 不 正常 這邊 來 => 
真的   主權 問題 不 敢生 小孩 覺得 有 點扯 => 
高雄市 隨便 丟 兩 張圖 出來 就 說 卡韓 。 => 
TMD 主席 不 是 柯屁 嗎 ?   不關 他 屁事 ? => 
柯韓 一起 下 吧   高雄市 整年 都 在 搞 選舉 自稱 務實 的 市長 在 護航   務實 在 哪 ？ 就 政治 算計 而已 => 
你看 職棒 簽賭 的 組頭 不 都 藍營 的 嗎 => 2, 8
https : / / i . imgur . com / BIlyq5q . jpg => 
反正 我 不會 投 你 => 12
紅利 用完 了     中共 經濟 專家 都 說 保 4 爭 5 了 明年 保 4 爭 5     勢在必行 => 
柯 鎮惡 的 風向 也 變換 得 太 快 了 => 6, 10
不 穩 才 有加 錢 ， 賺錢 領導 越維越 不 穩 => 
是 購物 旅行 ... 導遊 會 帶 你們 去 買買 買 但是 最後一天   要包 大 紅包 塞給 導遊 和 司機 ZZ 是 我 親戚 的 經驗   以前 常去   後 來近 幾年 不再 去 => 
是不是 要 開除黨籍 => 10
阿諾 人 呢 => 
還好 我 的 醫材 都 銷美 歐   蚵蚵 簡單 啊   TSMC 東西 +1 00%  

In [66]:
# using MLkNN
print("using MLkNN (k=20)")
from skmultilearn.adapt import MLkNN
classifier = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MLkNN(k=20))])
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)
#print(predictions)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))

all_labels = mb.inverse_transform(predictions)
for item, labels in zip(X_test, all_labels):
    print('{0} => {1}'.format(str(item,'utf-8'), ', '.join('%s' %id for id in labels)))

using MLkNN (k=20)
Accuracy =  0.654
不演了 不演了   政治 清流 白色力量 ww => 2, 4
說個笑話 ： 睡菊 是 五星市長 … 大菊 為重 => 
 => 
回收黨 看上 中 國瑜 了   有趣 => 
不 是 一 堆 柯糞 聽到 柯韓一家親 就 馬上 說 蟑螂 硬要 把 柯 跟 韓放 一起 講 很 噁心 嗎 柯糞 自己 去死 一 死好 嗎 zz => 
導遊 部 分 薪水 就是 從 購物 站 抽成 => 
連 這 種 都 賴帳 真的很 廢 => 10
這樣 也 算 切割 ?   直接 講 我 不 認得 她 不 是 更 猛 => 
雖 敗猶榮 => 
若 國民黨 繼續執政 ， 終有一天 台灣 會 被 統一 => 
推 => 
舔共 的 還 在 台灣 趴 趴 走   才 悲哀 吧   ： ） => 
發現 民眾黨 往 正常 方向 移動 ， 只好 三角形 拉往 不 正常 這邊 來 => 
真的   主權 問題 不 敢生 小孩 覺得 有 點扯 => 
高雄市 隨便 丟 兩 張圖 出來 就 說 卡韓 。 => 
TMD 主席 不 是 柯屁 嗎 ?   不關 他 屁事 ? => 
柯韓 一起 下 吧   高雄市 整年 都 在 搞 選舉 自稱 務實 的 市長 在 護航   務實 在 哪 ？ 就 政治 算計 而已 => 
你看 職棒 簽賭 的 組頭 不 都 藍營 的 嗎 => 
https : / / i . imgur . com / BIlyq5q . jpg => 
反正 我 不會 投 你 => 
紅利 用完 了     中共 經濟 專家 都 說 保 4 爭 5 了 明年 保 4 爭 5     勢在必行 => 
柯 鎮惡 的 風向 也 變換 得 太 快 了 => 
不 穩 才 有加 錢 ， 賺錢 領導 越維越 不 穩 => 
是 購物 旅行 ... 導遊 會 帶 你們 去 買買 買 但是 最後一天   要包 大 紅包 塞給 導遊 和 司機 ZZ 是 我 親戚 的 經驗   以前 常去   後 來近 幾年 不再 去 => 
是不是 要 開除黨籍 => 
阿諾 人 呢 => 
還好 我 的 醫材 都 銷美 歐   蚵蚵 簡單 啊   TSMC 東西 +1 00%   蚵蚵 聽說 TSMC 產能 大家 搶   大陸 不要   就 